In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import scale
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

# Preparing Data

In [12]:
X = np.random.rand(100, 10)
X_scaled = scale(X, axis=1)

In [13]:
y = (10 * X[:, 0] + 5 * X[:, 1] + 0.1 * X[:, 2] + np.random.normal(loc=0, scale=1, size=1)).reshape(100,1)

# Least Squares Linear Regression

In [14]:
clf = LinearRegression()
clf.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [15]:
clf.coef_

array([[  1.00000000e+01,   5.00000000e+00,   1.00000000e-01,
          1.33226763e-15,  -2.22044605e-16,  -1.11022302e-16,
          1.99840144e-15,   1.77635684e-15,  -7.77156117e-16,
          5.82867088e-16]])

# Lasso

In [16]:
def lasso(alpha, X, y):
    clf_lasso = Lasso(alpha=alpha)
    clf_lasso.fit(X, y)
    return clf_lasso.coef_

In [24]:
alphas = [0.001, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]

d = []
for alpha in alphas:
    coeffs = lasso(alpha, X, y)
    d.append({'alpha': alpha, 'X1': coeffs[0], 'X2': coeffs[1]})

df_lasso = pd.DataFrame(d, columns=['alpha', 'X1', 'X2'])

In [18]:
df_lasso

,alpha,X1,X2
0,0.001,2.568509,1.523523
1,0.010,2.566787,1.523394
2,0.050,2.532868,1.499323
3,0.100,2.477048,1.448676
4,0.150,2.425399,1.398440
5,0.200,2.372635,1.345839
6,0.250,2.317116,1.287391
7,0.300,2.261597,1.228944


# Ridge

In [19]:
def ridge(alpha, X, y):
    clf_ridge = Ridge(alpha)
    clf_ridge.fit(X, y)
    return clf_ridge.coef_    

In [22]:
alphas = [0.001, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]

d = []
for alpha in alphas:
    coeffs = ridge(alpha, X, y)
    d.append({'alpha': alpha, 'X1': coeffs[0][0], 'X2': coeffs[0][1]})

df_ridge = pd.DataFrame(d, columns=['alpha', 'X1', 'X2'])

In [23]:
df_ridge

,alpha,X1,X2
0,0.001,9.998681,4.999422
1,0.010,9.986828,4.994227
2,0.050,9.934518,4.971298
3,0.100,9.869971,4.942993
4,0.150,9.806336,4.915076
5,0.200,9.743593,4.887536
6,0.250,9.681723,4.860364
7,0.300,9.620705,4.833553
